In [1]:
# System
import os
from dotenv import load_dotenv
import ast

# LLM Models
from langchain_openai import ChatOpenAI
# from openai import OpenAI

# Template
from langchain_core.prompts import ChatPromptTemplate

# OutputParsers
from langchain.schema.output_parser import StrOutputParser

# Gradio frontend
import gradio as gr

In [2]:
load_dotenv()

True

In [3]:
chat_model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                        max_completion_tokens=8192,
                        api_key=os.getenv("OPENAI_API_KEY"),
                        temperature=0.8)

In [4]:
questions = [
    {"Question": "Question 1", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 2", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 3", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 4", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 5", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 6", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 7", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 8", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 9", "Choices": ["A", "B", "C", "D"], "Answer": "A"},
    {"Question": "Question 10", "Choices": ["A", "B", "C", "D"], "Answer": "A"}
]

In [5]:
def extract_questions(content):
    result = list()
    questions = content.split('---')
    counter = 1

    for ques in questions:
        parts = ques.split('\n')
        q = c = a = ""
        for part in parts:
            if part.startswith('Question:'):
                q = f'Question {counter}: ' + part.split('Question: ')[1].rstrip(' ')
            elif part.startswith('Choices:'):
                c = ast.literal_eval(part.split('Choices: ')[1].rstrip(' '))
            elif part.startswith('Answer:'):
                a = part.split('Answer: ')[1].rstrip(' ').rstrip('"').lstrip('"')
        if q != "":
            result.append({"Question": q, "Choices": c, "Answer": a})
            counter += 1

    return result

In [6]:
def question_generator(subject, grade):

    prompt = ChatPromptTemplate.from_template(
        """
        Generate a multiple-choice exam with 10 questions 
        on {subject} for a {grade} grade level student.
        Structure each question in the following format:
        Question: What is the capital of France?
        Choices: ["London", "Paris", "Rome", "Berlin"]
        Answer: "Paris"
        Separate each question by '---'.
        Do not surround 'Question' with double asterisk '**'.
        """
    )

    chat_chain = prompt | chat_model | StrOutputParser()
    response = chat_chain.invoke({"subject": subject, "grade": grade})

    return response

In [7]:
def exam_generator2(subject, grade):

    global questions

    questions_text = question_generator(subject, grade)
    questions = extract_questions(questions_text)
    inputs = []

    for question in questions:
        inputs.append(gr.Radio(choices=question["Choices"], value=None, label=question["Question"]))

    return inputs

In [8]:
def grade_exam(*answers):
    score = 0
    for i, question in enumerate(questions):
        if answers[i] not in question['Choices']:
            score += 0
        elif answers[i] == question["Answer"]:
            score += 1
    total_questions = len(questions)
    return f"You scored {score} out of {total_questions}."

In [9]:
css_template = """                   
               h1 {
                   font-size: 36px;
                   font-family: Verdana, Arial, Sans-Serif;
                   font-style: normal;
                   font-weight: bold;
                   # color: black; # rgb(238, 130, 238)
                   # background-color: yellow;
                   text-align: center;
                   text-transform: normal;
                   letter-spacing: 2px;
                   # border: 1px solid lightblue;
                   # border-radius: 0px
                   }

               p {
                   font-size: 14px;
                   font-family: Times New Roman, Times;
                   font-style: normal;
                   font-weight: normal;
                   # color: black; # rgb(238, 130, 238)
                   # background-color: yellow;
                   text-align: left;
                   text-transform: normal;
                   letter-spacing: 0px;
                   # border: 1px solid lightblue;
                   # border-radius: 0px
                   }
               """

In [10]:
with gr.Blocks(css=css_template) as iface:
    gr.Markdown("# Exam Generator")
    gr.Markdown(
        "This app generates an exam on any subject matter at a selected grade level. "
        "Type in the subject or select one from the dropdown menu. "
        "Select the grade level and click on 'Generate New Exam Question'. "
        "After selecting the most appropriate choices, click on Grade Exam to see your score.")
    with gr.Row():
        dropdown1 = gr.Dropdown(choices=['Math',
                                         'Geography',
                                         'U.S. History',
                                         'Science',
                                         'History'],
                                value='History',
                                multiselect=False,
                                allow_custom_value=True,
                                label="Subject")
        dropdown2 = gr.Dropdown(choices=['First grade',
                                         'Second grade',
                                         'Third grade',
                                         'Fourth grade',
                                         'Fifth grade',
                                         'Sixth grade',
                                         'Junior High School',
                                         'High School',
                                         'Undergraduate',
                                         'Master degree',
                                         'Doctorate',
                                         'Intermediate',
                                         'Advanced',
                                         'Expert'],
                                value='Third grade',
                                multiselect=False,
                                allow_custom_value=True,
                                label="Exam Grade Level")

    update_btn1 = gr.Button("Generate New Exam Questions")

    inputs = []
    for question in questions:
        inputs.append(gr.Radio(choices=question["Choices"], label=question["Question"]))

    update_btn1.click(fn=exam_generator2,
                      inputs=[dropdown1, dropdown2],
                      outputs=inputs)

    submit_btn = gr.Button("Grade Exam")

    output2 = gr.Textbox(label="Exam Result")
    submit_btn.click(fn=grade_exam, inputs=inputs, outputs=output2)

In [11]:
if __name__ == '__main__':
    iface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [12]:
iface.close()

Closing server running on port: 7860
